In [44]:
## re-run the rejection reason final model after that test
from cashe import get_train_test_split, get_training_inputs, drop_nomodel_columns
from src.model_app import ModelApplication, ModelApplicationDeployment
from src.utilities_rejection import RejectionReasonLabeling
from src.data_local import MergedDataPreprocessing
from src.model_train import ModelTrainer
import pandas as pd
import numpy as np

df_train, df_test = get_train_test_split('data/HJH/12-06-2024')
df_train = df_train[df_train['OUTCOME'] !='APPROVED']
df_test = df_test[df_test['OUTCOME'] !='APPROVED']

train_rej = RejectionReasonLabeling(df_train)
train_rej.recoginze_label()
train_rej = RejectionReasonLabeling(df_test)
train_rej.recoginze_label()

def analyze_error_code(error_value):

    df_train['NPHIES_LABEL'] = np.where(df_train['NPHIES_CODE'] == error_value, 1, 0)
    df_test['NPHIES_LABEL']  = np.where( df_test['NPHIES_CODE']  == error_value, 1, 0)
    X_train, y_train, X_test, y_test = get_training_inputs(df_train, df_test)
    X_train = drop_nomodel_columns(X_train);  X_test = drop_nomodel_columns(X_test)
    X_train.drop(columns=['NPHIES_CODE'],inplace=True); X_test.drop(columns=['NPHIES_CODE'],inplace=True)

    preprocessing_train = MergedDataPreprocessing(X_train)
    X_train_prep = preprocessing_train.columns_prep(service_encoding=False)

    preprocessing_test = MergedDataPreprocessing(X_test)
    X_test_prep = preprocessing_test.columns_prep(service_encoding=False)

    X_train_encoded = preprocessing_train.column_embedding(X_train_prep)
    X_test_encoded = preprocessing_train.column_embedding(X_test_prep)

    X_train_encoded.fillna(0,inplace=True)
    X_test_encoded.fillna(0,inplace=True)

    model_runner = ModelTrainer(X_train=X_train_encoded,y_train=y_train['NPHIES_LABEL'],
                                X_test=X_test_encoded,y_test=y_test['NPHIES_LABEL'])

    model_runner.train_models()
    result_analysis = model_runner.evaluate_models()
    top_model_features = model_runner.get_xgboost_feature_importance()

    features_df = pd.DataFrame({"Column_name":X_test_encoded.columns,"Feature_importance":top_model_features}).sort_values(
        by=['Feature_importance'],ascending=False)
    model_features = find_all_importances(features_df)
    return result_analysis['XGBoost'], model_features

def find_all_importances(model_features):
    combined_text_rows = model_features[model_features['Column_name'].str.contains('CombinedText')]
    combined_sum = sum(combined_text_rows['Feature_importance'])
    model_features = model_features[~model_features['Column_name'].str.contains('CombinedText')]
    model_features.loc[len(model_features)] = {'Column_name':'SERVICE_DESCRIPTION','Feature_importance':float(combined_sum)}
    model_features.sort_values(by=['Feature_importance'],ascending=False,inplace=True)
    return model_features

In [45]:
error_value = 'MN-1-1'
model_performance, model_features = analyze_error_code(error_value)

E:\Projects\Claims_Rejection\src\data_local.py:269: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  self.df['PatientAgeRange'] = self.df.PatientAgeRange.replace(age_encoding)
E:\Projects\Claims_Rejection\src\data_local.py:269: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  self.df['PatientAgeRange'] = self.df.PatientAgeRange.replace(age_encoding)


[LightGBM] [Info] Number of positive: 17007, number of negative: 62682
[LightGBM] [Info] Total Bins 5683
[LightGBM] [Info] Number of data points in the train set: 79689, number of used features: 47
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.213417 -> initscore=-1.304449
[LightGBM] [Info] Start training from score -1.304449


LightGBM, Decision Tree, SGD and Neural Network are trained on dataset.


C:\Users\omar.mounir\AppData\Local\Temp\ipykernel_8260\3176289654.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  model_features.loc[len(model_features)] = {'Column_name':'SERVICE_DESCRIPTION','Feature_importance':float(combined_sum)}
C:\Users\omar.mounir\AppData\Local\Temp\ipykernel_8260\3176289654.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  model_features.sort_values(by=['Feature_importance'],ascending=False,inplace=True)


In [46]:
print(model_performance)

{'Accuracy': 0.9, 'Precision': 0.83, 'Recall': 0.67, 'F1 Score': 0.74}


In [47]:
print(model_features)

                 Column_name  Feature_importance
33       SERVICE_DESCRIPTION            0.263460
6             VAT_PERCENTAGE            0.160544
29            PURCHASER_CODE            0.080892
27           DEPARTMENT_TYPE            0.052364
14  PROVIDER_DEPARTMENT_CODE            0.042784
28            TREATMENT_TYPE            0.040586
10    UNIT_PRICE_STOCKED_UOM            0.035059
12       DISCOUNT_PERCENTAGE            0.034723
8                   DURATION            0.024076
19       PATIENT_NATIONALITY            0.023381
2    LINE_CLAIMED_AMOUNT_SAR            0.021000
4             NET_VAT_AMOUNT            0.017802
11            UNIT_PRICE_NET            0.017312
30                  NEW_BORN            0.017070
0               NET_WITH_VAT            0.015743
25          TOTAL_DEDUCTIBLE            0.015454
3         LINE_ITEM_DISCOUNT            0.013718
1                        QTY            0.013328
15     DOCTOR_SPECIALTY_CODE            0.012998
24            TOTAL_

In [54]:
error_value = 'CV-1-1'
model_performance, model_features = analyze_error_code(error_value)

E:\Projects\Claims_Rejection\src\data_local.py:269: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  self.df['PatientAgeRange'] = self.df.PatientAgeRange.replace(age_encoding)
E:\Projects\Claims_Rejection\src\data_local.py:269: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  self.df['PatientAgeRange'] = self.df.PatientAgeRange.replace(age_encoding)


[LightGBM] [Info] Number of positive: 12461, number of negative: 67228
[LightGBM] [Info] Total Bins 5683
[LightGBM] [Info] Number of data points in the train set: 79689, number of used features: 47
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.156370 -> initscore=-1.685486
[LightGBM] [Info] Start training from score -1.685486


LightGBM, Decision Tree, SGD and Neural Network are trained on dataset.


C:\Users\omar.mounir\AppData\Local\Temp\ipykernel_8260\3176289654.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  model_features.loc[len(model_features)] = {'Column_name':'SERVICE_DESCRIPTION','Feature_importance':float(combined_sum)}
C:\Users\omar.mounir\AppData\Local\Temp\ipykernel_8260\3176289654.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  model_features.sort_values(by=['Feature_importance'],ascending=False,inplace=True)


In [57]:
model_performance

{'Accuracy': 0.92, 'Precision': 0.73, 'Recall': 0.81, 'F1 Score': 0.77}

In [58]:
model_features

,Column_name,Feature_importance
29,PURCHASER_CODE,0.307840
28,TREATMENT_TYPE,0.114707
27,DEPARTMENT_TYPE,0.109626
19,PATIENT_NATIONALITY,0.091035
33,SERVICE_DESCRIPTION,0.059344
13,EMERGENCY_INDICATOR,0.054303
12,DISCOUNT_PERCENTAGE,0.052148
15,DOCTOR_SPECIALTY_CODE,0.020894
24,TOTAL_DISCOUNT,0.018247
23,TOTAL_CLAIMED_AMOUNT_SAR,0.016802


In [51]:
error_value = 'SE-1-6'
model_performance, model_features = analyze_error_code(error_value)

E:\Projects\Claims_Rejection\src\data_local.py:269: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  self.df['PatientAgeRange'] = self.df.PatientAgeRange.replace(age_encoding)
E:\Projects\Claims_Rejection\src\data_local.py:269: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  self.df['PatientAgeRange'] = self.df.PatientAgeRange.replace(age_encoding)


[LightGBM] [Info] Number of positive: 10171, number of negative: 69518
[LightGBM] [Info] Total Bins 5683
[LightGBM] [Info] Number of data points in the train set: 79689, number of used features: 47
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.127634 -> initscore=-1.922045
[LightGBM] [Info] Start training from score -1.922045


LightGBM, Decision Tree, SGD and Neural Network are trained on dataset.


C:\Users\omar.mounir\AppData\Local\Temp\ipykernel_8260\3176289654.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  model_features.loc[len(model_features)] = {'Column_name':'SERVICE_DESCRIPTION','Feature_importance':float(combined_sum)}
C:\Users\omar.mounir\AppData\Local\Temp\ipykernel_8260\3176289654.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  model_features.sort_values(by=['Feature_importance'],ascending=False,inplace=True)


In [53]:
print(model_performance)

{'Accuracy': 0.96, 'Precision': 0.92, 'Recall': 0.77, 'F1 Score': 0.84}


In [52]:
model_features

,Column_name,Feature_importance
29,PURCHASER_CODE,0.506960
33,SERVICE_DESCRIPTION,0.074825
19,PATIENT_NATIONALITY,0.068148
26,TOTAL_PATIENT_VATAMOUNT,0.033929
1,QTY,0.030968
27,DEPARTMENT_TYPE,0.022640
15,DOCTOR_SPECIALTY_CODE,0.020823
23,TOTAL_CLAIMED_AMOUNT_SAR,0.018652
25,TOTAL_DEDUCTIBLE,0.018374
2,LINE_CLAIMED_AMOUNT_SAR,0.015629


In [59]:
error_value = 'SE-1-8'
model_performance, model_features = analyze_error_code(error_value)

E:\Projects\Claims_Rejection\src\data_local.py:269: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  self.df['PatientAgeRange'] = self.df.PatientAgeRange.replace(age_encoding)
E:\Projects\Claims_Rejection\src\data_local.py:269: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  self.df['PatientAgeRange'] = self.df.PatientAgeRange.replace(age_encoding)


[LightGBM] [Info] Number of positive: 1297, number of negative: 78392
[LightGBM] [Info] Total Bins 5683
[LightGBM] [Info] Number of data points in the train set: 79689, number of used features: 47
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.016276 -> initscore=-4.101668
[LightGBM] [Info] Start training from score -4.101668


LightGBM, Decision Tree, SGD and Neural Network are trained on dataset.


E:\Projects\Claims_Rejection\src\model_train.py:59: RuntimeWarning: invalid value encountered in scalar divide
  mod_f1 = (2*mod_precision*mod_recall) / (mod_precision+mod_recall)
E:\Projects\Claims_Rejection\venv\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
E:\Projects\Claims_Rejection\src\model_train.py:59: RuntimeWarning: invalid value encountered in scalar divide
  mod_f1 = (2*mod_precision*mod_recall) / (mod_precision+mod_recall)
C:\Users\omar.mounir\AppData\Local\Temp\ipykernel_8260\3176289654.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  model_feature

In [60]:
model_performance ## data is extremely balanced, predictive analytics couldn't be done

{'Accuracy': 0.98, 'Precision': 0.14, 'Recall': 0.04, 'F1 Score': 0.06}

In [61]:
model_features

,Column_name,Feature_importance
27,DEPARTMENT_TYPE,0.447325
29,PURCHASER_CODE,0.123270
33,SERVICE_DESCRIPTION,0.047021
19,PATIENT_NATIONALITY,0.046959
14,PROVIDER_DEPARTMENT_CODE,0.042484
24,TOTAL_DISCOUNT,0.040729
16,DOCTOR_CODE,0.036699
15,DOCTOR_SPECIALTY_CODE,0.029574
23,TOTAL_CLAIMED_AMOUNT_SAR,0.028882
17,PATIENT_AGE,0.026865
